In [1]:
!pip install --upgrade numpy
!pip install --upgrade pandas scikit-learn openpyxl tensorflow

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have

In [12]:
from google.colab import files
uploaded = files.upload()
# Luego asigna el nombre de la clave al path:
excel_path = list(uploaded.keys())[0]

Saving Multimodal summary metrics (Excel).xlsx to Multimodal summary metrics (Excel) (2).xlsx


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer

# Ruta al archivo Excel
excel_path = 'Multimodal summary metrics (Excel).xlsx'

# 1. Carga tus datos
df = pd.read_excel(
    excel_path,
    sheet_name='Hoja1',
    engine='openpyxl',
    usecols=['Respondent Name', 'Terror', 'Destreza', 'Workout', 'Picos']
)
# Inspecciona nombres de columnas para confirmar lectura correcta:
print("Columnas disponibles tras usecols:", df.columns.tolist())
# Elimina espacios en los nombres de columna si los hubiera
df.columns = df.columns.str.strip()

# 2. Prepara características y objetivo. Prepara características y objetivo Prepara características y objetivo
y = df['Picos'].values
X = df[['Terror', 'Destreza', 'Workout']].values

# 3. Escala las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Divide en entrenamiento/validación
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 5. Construye el modelo secuencial
def build_sequential_model(input_dim):
    model = Sequential([
        InputLayer(input_shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    return model

model = build_sequential_model(X_train.shape[1])
model.summary()

# 6. Entrena el modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32
)

# 7. Evalúa desempeño
test_loss, test_mae = model.evaluate(X_val, y_val)
print(f"MSE en validación: {test_loss:.4f}")
print(f"MAE en validación: {test_mae:.4f}")

# 8. Guarda el modelo
model.save('modelo_categorias_continuo.h5')

ValueError: Usecols do not match columns, columns expected but not found: ['Picos'] (sheet: Hoja1)